# How to Parse Raw ACS Data
The purpose of this notebook is to parse American Community Survey data. As stated on the United States Census Bureau site, “The 5-year estimates from the ACS are "period" estimates that represent data collected over a period of time. The primary advantage of using multiyear estimates is the increased statistical reliability of the data for less populated areas and small population subgroups." This example will parse ACS data from Alabama and return each parsed table in a dictionary.

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
from os.path import join, expanduser, dirname
import pandas as pd
import sys
import os
import re
import warnings

In [3]:
warnings.filterwarnings(action='ignore')
home = expanduser('~')

src_path = '{}/zrp'.format(home)
sys.path.append(src_path)

In [4]:
from zrp.prepare import ProcessStrings


/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Predefine paths & required parameters

In [4]:
# Support files path pointing to where the raw ACS data is stored
support_files_path = "INSERT-PATH-HERE"
# Year of ACS data
year = "2019"
# Span of ACS data. The ACS data is available in 1 or 5 year spans. 
span = "5"
# State
state_level = "al"
# State County FIPs Code
st_cty_code = "01001"

Import ACS Lookup Functions

In [9]:
from zrp.prepare.acs_lookup import *


### Initialize `ACS_Parser`
This class constructs American Community Survey lookup tables that enables race approximation. Census American Community Survey data is required for this module to run. You can retrieve 2019 data from
- https://www2.census.gov/programs-surveys/acs/summary_file/2019/data/2019_5yr_Summary_FileTemplates.zip
- https://www2.census.gov/programs-surveys/acs/summary_file/2019/data/5_year_entire_sf/2019_ACS_Geography_Files.zip
- https://www2.census.gov/programs-surveys/acs/summary_file/2019/data/5_year_entire_sf/All_Geographies_Not_Tracts_Block_Groups.zip
- https://www2.census.gov/programs-surveys/acs/summary_file/2019/data/5_year_entire_sf/Tracts_Block_Groups_Only.zip


In [10]:
acs_parse = ACS_Parser(support_files_path = support_files_path, year = year, span = span, state_level = state_level, n_jobs=-1 )

### Run `ACS_Parser`
Lookup tables are saved by default. 

In [11]:
%%time 
output = acs_parse.transform(save_table = False)


  0%|          | 0/141 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 15 concurrent workers.

 32%|███▏      | 45/141 [01:25<02:18,  1.44s/it][Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  1.4min

100%|██████████| 141/141 [05:26<00:00,  2.32s/it]


CPU times: user 16.4 s, sys: 5.07 s, total: 21.5 s
Wall time: 6min 54s


[Parallel(n_jobs=-1)]: Done 141 out of 141 | elapsed:  6.9min finished


### Preview 

In [12]:
output.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141'])

In [13]:
output['1']['data'].head()
print(output['1']['data'].shape)

(8857, 244)


In [14]:
output['100']['data'].head()
print(output['100']['data'].head())

  FILEID FILETYPE STUSAB CHARITER SEQUENCE LOGRECNO B24123_246 B24123_247  \
0  ACSSF   2019e5     al      000     0100  0001772        NaN        NaN   
1  ACSSF   2019e5     al      000     0100  0001773        NaN        NaN   
2  ACSSF   2019e5     al      000     0100  0001774        NaN        NaN   
3  ACSSF   2019e5     al      000     0100  0001775        NaN        NaN   
4  ACSSF   2019e5     al      000     0100  0001776        NaN        NaN   

  B24123_248 B24123_249  ... B24123_484 B24123_485 B24123_486 B24123_487  \
0        NaN        NaN  ...        NaN        NaN        NaN        NaN   
1        NaN        NaN  ...        NaN        NaN        NaN        NaN   
2        NaN        NaN  ...        NaN        NaN        NaN        NaN   
3        NaN        NaN  ...        NaN        NaN        NaN        NaN   
4        NaN        NaN  ...        NaN        NaN        NaN        NaN   

  B24123_488 B24123_489 B24123_490 State               GEOID  \
0        NaN    